# V2 Notebook 4: The Robust MPC Core: Integrating Intelligence

**Project:** `RobustMPC-Pharma` (V2)
**Goal:** To assemble our advanced components into the final `RobustMPCController`. This new controller core will be uncertainty-aware, adaptive to disturbances, and capable of sophisticated optimization. This is where all our hard work comes together.

### Table of Contents
1. [Theory: The Pillars of a Robust Controller](#1.-Theory:-The-Pillars-of-a-Robust-Controller)
2. [Implementing the `RobustMPCController` V2](#2.-Implementing-the-RobustMPCController-V2)
3. [Standalone Test of the Integrated Controller](#3.-Standalone-Test-of-the-Integrated-Controller)

--- 
## 1. Theory: The Pillars of a Robust Controller

Our V2 controller is built on three pillars that address the weaknesses of the V1 prototype:

1.  **Stable State Perception (The Kalman Filter):** It will not react to raw sensor noise. Instead, it will act on a smooth, stable estimate of the true process state. This prevents control jitter and improves efficiency.

2.  **Risk-Aware Decision Making (The Probabilistic Model):** The controller's cost function will not just evaluate the model's mean prediction. It will incorporate the model's uncertainty (standard deviation). By optimizing a **risk-adjusted** forecast (e.g., Upper Confidence Bound), the controller will act more cautiously when the model is uncertain, making it fundamentally safer.

3.  **Adaptability to Disturbances (Integral Action):** Real-world processes suffer from un-modeled, persistent disturbances (e.g., gradual equipment fouling, changes in raw material). A standard controller will have a steady-state error in these cases. By implementing **Integral Action**, our controller will learn to recognize and automatically compensate for these disturbances, ensuring it always drives the process precisely to the target setpoint. This is known as **Offset-Free MPC**.

--- 
## 2. Implementing the `RobustMPCController` V2

We will now create the final `RobustMPCController` class in `src/core.py`. This class will be a high-level orchestrator, managing the interactions between the state estimator, the predictive model, and the optimizer.

The core logic will follow this sequence:
1.  Receive a noisy measurement from the plant.
2.  Use the `KalmanStateEstimator` to get a clean state estimate.
3.  Update its internal `disturbance_estimate` using the error between the setpoint and the clean state (Integral Action).
4.  Define a `fitness_function` for the optimizer. This function will:
    a. Take a candidate control plan.
    b. Get a probabilistic prediction from the `ProbabilisticTransformer`.
    c. Add the `disturbance_estimate` to the mean prediction.
    d. Calculate a risk-adjusted cost using the mean and standard deviation.
5.  Pass this fitness function to the `GeneticOptimizer` to find the best control plan.
6.  Return the first step of the winning plan.

In [2]:
%%writefile ../robust_mpc/core.py
import numpy as np
import torch

class RobustMPCController:
    """
    An advanced MPC controller that integrates a state estimator,
    a probabilistic model, and a genetic algorithm optimizer for robust,
    offset-free, and uncertainty-aware control.
    """
    def __init__(self, model, estimator, optimizer_class, config, scalers):
        self.model = model
        self.estimator = estimator
        self.optimizer_class = optimizer_class
        self.config = config
        self.scalers = scalers
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)

        # Initialize the disturbance estimate for Integral Action
        self.disturbance_estimate = np.zeros(len(config['cma_names']))

    def _update_disturbance_estimate(self, smooth_state, setpoint):
        """Updates the integral error term for offset-free control."""
        error = setpoint - smooth_state
        # The gain (alpha) determines how quickly the controller adapts to the disturbance
        alpha = self.config.get('integral_gain', 0.1)
        self.disturbance_estimate += alpha * error

    def _get_fitness_function(self, past_cmas_scaled, past_cpps_scaled, target_cmas_unscaled):
        """
        Creates and returns the fitness function to be used by the GA.
        This function captures the current state and target.
        """
        def fitness(control_plan_unscaled):
            # --- 1. Prepare Inputs ---
            # Scale the unscaled control plan generated by the GA
            plan_scaled = self._scale_cpp_plan(control_plan_unscaled)
            
            # Convert all inputs to tensors
            past_cmas_tensor = torch.tensor(past_cmas_scaled, dtype=torch.float32).unsqueeze(0).to(self.device)
            past_cpps_tensor = torch.tensor(past_cpps_scaled, dtype=torch.float32).unsqueeze(0).to(self.device)
            future_cpps_tensor = torch.tensor(plan_scaled, dtype=torch.float32).unsqueeze(0).to(self.device)

            # --- 2. Get Probabilistic Prediction ---
            mean_pred_scaled, std_pred_scaled = self.model.predict_distribution(
                past_cmas_tensor, past_cpps_tensor, future_cpps_tensor, 
                n_samples=self.config.get('mc_samples', 30)
            )

            # --- 3. Calculate Risk-Adjusted, Corrected Prediction ---
            # Correct for disturbance (Integral Action)
            disturbance_scaled = self._scale_cma_vector(self.disturbance_estimate)
            corrected_mean_scaled = mean_pred_scaled + torch.tensor(disturbance_scaled, device=self.device)
            
            # Adjust for risk (Uncertainty-Awareness)
            beta = self.config.get('risk_beta', 1.5) # Higher beta = more cautious
            # For minimization, we penalize the upper bound of the error
            risk_adjusted_pred_scaled = corrected_mean_scaled + beta * std_pred_scaled

            # --- 4. Calculate Cost ---
            target_scaled = self._scale_cma_plan(target_cmas_unscaled)
            target_tensor = torch.tensor(target_scaled, dtype=torch.float32).to(self.device)
            cost = torch.mean(torch.abs(risk_adjusted_pred_scaled - target_tensor))

            return cost.item()
        
        return fitness
        
    def suggest_action(self, noisy_measurement, control_input, setpoint):
        # 1. Get a clean state estimate
        smooth_state = self.estimator.estimate(noisy_measurement, control_input)

        # 2. Update the integral error term
        self._update_disturbance_estimate(smooth_state, setpoint)

        # 3. Create the fitness function for this specific time step
        # This part requires getting the historical data, which we will simulate for the test.
        # In a real app, this would come from a data buffer.
        past_cmas_scaled, past_cpps_scaled = self._get_scaled_history(smooth_state)
        
        # The target is the setpoint repeated over the horizon
        target_plan = np.tile(setpoint, (self.config['horizon'], 1))
        fitness_func = self._get_fitness_function(past_cmas_scaled, past_cpps_scaled, target_plan)

        # 4. Instantiate and run the optimizer
        param_bounds = self._get_param_bounds()
        optimizer = self.optimizer_class(fitness_func, param_bounds, self.config['ga_config'])
        best_plan = optimizer.optimize()

        # 5. Return the first step of the optimal plan
        return best_plan[0]

    # --- Helper methods for scaling and data management ---
    def _get_scaled_history(self, current_smooth_state):
        # In a real app, this would pull from a historical data buffer.
        # For this test, we'll create dummy history.
        L = self.config['lookback']
        past_cmas_unscaled = np.tile(current_smooth_state, (L, 1))
        past_cpps_unscaled = np.tile([120, 500, 30], (L, 1)) # Dummy CPPs
        # Add soft sensors to CPPs
        # ... (logic from notebook V1-2) ...
        # Scale both
        past_cmas_scaled = self._scale_cma_plan(past_cmas_unscaled)
        past_cpps_scaled = self._scale_cpp_plan(past_cpps_unscaled, with_soft_sensors=True)
        return past_cmas_scaled, past_cpps_scaled

    def _scale_cpp_plan(self, plan_unscaled, with_soft_sensors=False):
        # This needs to be implemented robustly, matching the training preprocessing
        # For now, a placeholder
        if with_soft_sensors: return np.zeros((plan_unscaled.shape[0], len(self.config['cpp_full_names'])))
        return np.zeros_like(plan_unscaled)
        
    def _scale_cma_plan(self, plan_unscaled):
        return np.zeros_like(plan_unscaled)

    def _scale_cma_vector(self, vector_unscaled):
        return np.zeros_like(vector_unscaled)
        
    def _get_param_bounds(self):
        param_bounds = []
        cpp_config = self.config['cpp_constraints']
        for _ in range(self.config['horizon']):
            for name in self.config['cpp_names']:
                param_bounds.append((cpp_config[name]['min_val'], cpp_config[name]['max_val']))
        return param_bounds

Overwriting ../robust_mpc/core.py


--- 
## 3. Standalone Test of the Integrated Controller

Before the final showdown in Notebook 5, let's do a quick standalone test of the new `RobustMPCController` class. This will ensure all the components are communicating correctly. We will mock the necessary inputs and call the main `.suggest_action()` method to see if it produces a sensible result.

In [3]:
# This cell is for testing the class logic. A full implementation requires all components.
# Due to the complexity of mocking all inputs (history, scalers, etc.), a full test is deferred to Notebook 5.
# Here we will just instantiate the class to check for syntax errors.

from V2.robust_mpc.core import RobustMPCController
from V2.robust_mpc.estimators import KalmanStateEstimator
from V2.robust_mpc.models import ProbabilisticTransformer
from V2.robust_mpc.optimizers import GeneticOptimizer

# --- Mock Components and Configs (for instantiation) ---
mock_model = ProbabilisticTransformer(cma_features=2, cpp_features=5, d_model=32, nhead=2)
mock_estimator = 'KalmanFilterPlaceholder'
mock_optimizer_class = GeneticOptimizer
mock_scalers = 'ScalersPlaceholder'

MPC_CONFIG_V2 = {
    'lookback': 36,
    'horizon': 10, # Shorter for faster testing
    'cma_names': ['d50', 'lod'],
    'cpp_names': ['spray_rate', 'air_flow', 'carousel_speed'],
    'cpp_full_names': ['spray_rate', 'air_flow', 'carousel_speed', 'specific_energy', 'froude_number_proxy'],
    'integral_gain': 0.1,
    'risk_beta': 1.5,
    'mc_samples': 10,
    'cpp_constraints': {
        'spray_rate': {'min_val': 80.0, 'max_val': 180.0},
        'air_flow': {'min_val': 400.0, 'max_val': 700.0},
        'carousel_speed': {'min_val': 20.0, 'max_val': 40.0}
    },
    'ga_config': {
        'population_size': 40,
        'num_generations': 15,
        'crossover_prob': 0.7,
        'mutation_prob': 0.2,
        'horizon': 10, # Must match outer horizon
        'num_cpps': 3
    }
}

try:
    controller = RobustMPCController(
        model=mock_model,
        estimator=mock_estimator,
        optimizer_class=mock_optimizer_class,
        config=MPC_CONFIG_V2,
        scalers=mock_scalers
    )
    print("RobustMPCController class instantiated successfully!")
    print("Note: The helper methods for scaling and history need to be fully implemented for the final test.")
except Exception as e:
    print(f"An error occurred during instantiation: {e}")

✅ RobustMPCController initialized successfully!
   - Model: ProbabilisticTransformer
   - Estimator: str
   - Optimizer: GeneticOptimizer
   - Horizon: 10, Risk β: 1.5
RobustMPCController class instantiated successfully!
Note: The helper methods for scaling and history need to be fully implemented for the final test.


### Final Analysis and Next Steps

We have successfully designed and implemented the architecture for our advanced `RobustMPCController`. This new core brings together all the state-of-the-art components we've developed in this V2 series. While we have only performed a basic instantiation test here, we have laid out the complete logic that will be put to the test in our final notebook.

The key takeaways are:
*   **Modular Design:** The controller is built to accept different estimators, models, and optimizers, making it highly flexible.
*   **Integral Action:** The logic for `disturbance_estimate` is in place, ready to eliminate steady-state error.
*   **Risk-Awareness:** The fitness function is designed to use the probabilistic output of our model, making the controller inherently safer.

In the final notebook, we will implement the missing helper methods, connect this controller to our plant, and perform a head-to-head showdown against our V1 controller to definitively prove its superior performance.